In [18]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from proj1_helpers import *
from helpers import *
from implementations import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [19]:
DATA_TRAIN_PATH = 'data/train.csv' 
y, x, ids = load_csv_data(DATA_TRAIN_PATH)

In [22]:
sanitized_y = y.reshape((y.shape[0], 1))

In [23]:
sanitized_x = x.copy()

na_values_x = sanitized_x == -999

print(np.sum(na_values_x))

for i, feature in enumerate(sanitized_x.T):
    na_values = feature == -999
    known_values = ~na_values
    
    mean = np.mean(feature[known_values])
    std = np.std(feature[known_values])
    
    sanitized_x[:, i] = sanitized_x[:, i] - mean
    sanitized_x[:, i] = sanitized_x[:, i] / std
    
    
sanitized_x[na_values_x] = 0
sanitized_x[:,5][:50]

1580052


array([-0.62125844,  0.        ,  0.        ,  0.        ,  0.        ,
       -0.21926102, -0.53665123,  0.        ,  0.        ,  0.        ,
        0.        , -0.29968666,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        , -0.01870612,  0.        ,
        0.        ,  0.        ,  1.63325174,  0.        , -0.14063324,
        0.        , -0.19070156,  1.21453303, -0.32085355,  0.        ,
        0.        ,  0.        ,  0.        , -0.24220062, -0.51448357,
        0.        ,  0.        ,  0.27107761,  0.        ,  0.        ,
       -0.73823707,  0.30071075,  0.        ,  0.        ,  0.        ])

In [25]:
w_init = np.random.rand(sanitized_x.shape[1], 1)
print(w_init.shape)
w, loss = least_squares_GD(sanitized_y, sanitized_x, np.random.rand(sanitized_x.shape[1], 1), 1000, 0.01, verbose=True)
print(w, loss)

(30, 1)
Iteration : 0 with loss [[10.47290029]] and gradient: [[ 1.76726282]
 [-0.29923856]
 [ 1.62254188]
 [ 3.41462974]
 [-0.02015559]
 [ 0.37110268]
 [-0.05416912]
 [-0.97019522]
 [ 2.26926176]
 [ 3.82400243]
 [ 0.11302892]
 [ 2.08562892]
 [ 0.09305065]
 [ 2.45555473]
 [ 0.29082197]
 [-0.01372632]
 [ 1.84004905]
 [ 0.49211275]
 [ 0.49752027]
 [ 2.45341829]
 [ 0.61857722]
 [ 3.69032119]
 [ 2.73002668]
 [ 2.00235552]
 [ 0.17163147]
 [ 0.01814922]
 [ 0.94874223]
 [ 0.03937312]
 [ 0.14073496]
 [ 3.53847364]]
Iteration : 500 with loss [[0.43524241]] and gradient: [[ 0.01626948]
 [ 0.01113735]
 [ 0.01517624]
 [ 0.01449179]
 [ 0.00495708]
 [-0.00133732]
 [ 0.02299805]
 [-0.02767768]
 [ 0.00788255]
 [-0.00813798]
 [ 0.0250116 ]
 [ 0.01870171]
 [ 0.03190589]
 [ 0.00428636]
 [-0.00992155]
 [ 0.01352211]
 [-0.03549615]
 [ 0.00817773]
 [ 0.01821398]
 [-0.01368888]
 [ 0.01381836]
 [ 0.00132249]
 [-0.01832004]
 [-0.01479997]
 [ 0.00517461]
 [ 0.03422558]
 [ 0.05160924]
 [ 0.00811202]
 [ 0.0584059

In [31]:
prediction = sanitized_x @ w
print(prediction)

prediction[prediction >= 0] = 1
prediction[prediction <= 0] = -1

print(prediction)

print(np.sum(prediction == sanitized_y)/sanitized_y.shape[0])

[[-0.25516513]
 [ 0.31391931]
 [-0.42809016]
 ...
 [-0.0349503 ]
 [ 0.41117694]
 [-0.33713466]]
[[-1.]
 [ 1.]
 [-1.]
 ...
 [-1.]
 [ 1.]
 [-1.]]
0.71338


In [ ]:
print(compute_accuracy(sanitized_y, sanitized_x, ))

Reduce dimensions and standardize the columns of X

In [61]:
x_train_red = x_train[:, [0, 1, 2, 3, 8, 9, 10, 11, 13, 19, 21, 22, 29]]
x_test_red = x_test[:, [0, 1, 2, 3, 8, 9, 10, 11, 13, 19, 21, 22, 29]]
x_train_red = x_train[:, [1,3,4,5,6,7,8,9,11,12,13,15,18,23,25,26,28,29]]
x_test_red = x_test[:, [1,3,4,5,6,7,8,9,11,12,13,15,18,23,25,26,28,29]]

x_train_red_norm = standardize(x_train_red)
x_test_red_norm = standardize(x_test_red)

Running logistic regression with different values of degrees (polynomial feature expansion), gamma and different initial weights

In [63]:
number_of_w = 3

for degree in range(1, 5):
    tx_train = build_poly_cross_2(x_train_red_norm)
    tx_test = build_poly_cross_2(x_test_red_norm)
    for gamma in np.array([0.001, 0.01, 0.05, 0.1, 0.5]):
        print("Running logistic regression with polynomial feature expansion with degree", degree, 
                 "and gamma", gamma,", with", number_of_w, "different initial weights.")
        for i in  range(number_of_w):
            w_init = np.random.rand(tx_train.shape[1], 1)
            w, loss = logistic_regression(y_train, tx_train, w_init, 5000, gamma)
            print(i,":")
            print("Accuracy on training set", compute_accuracy(y_train, tx_train, w))
            print("Accuracy on test set:, ", compute_accuracy(y_test, tx_test, w))

Running logistic regression with polynomial feature expansion with degree 1 and gamma 0.001 , with 3 different initial weights.
Gradient have norm of 2.27335467333e-12
0 :
Accuracy on training set 0.35636
Accuracy on test set:,  0.35708
Gradient have norm of 0.0
1 :
Accuracy on training set 0.337744
Accuracy on test set:,  0.338424
Gradient have norm of 0.0
2 :
Accuracy on training set 0.354424
Accuracy on test set:,  0.35568
Running logistic regression with polynomial feature expansion with degree 1 and gamma 0.01 , with 3 different initial weights.
Gradient have norm of 1.01966234571e-12
0 :
Accuracy on training set 0.544
Accuracy on test set:,  0.545136
Gradient have norm of 43.1214248776
1 :
Accuracy on training set 0.657896
Accuracy on test set:,  0.656768
Gradient have norm of 0.0
2 :
Accuracy on training set 0.362208
Accuracy on test set:,  0.363216
Running logistic regression with polynomial feature expansion with degree 1 and gamma 0.05 , with 3 different initial weights.
Grad

KeyboardInterrupt: 

## Generate predictions and save ouput in csv format for submission:


In [10]:
DATA_TEST_PATH = '' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [31]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

In [346]:
a = np.random.rand(30)
np.around(a)

array([1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0.,
       1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.])

In [347]:
a

array([0.98948792, 0.67934654, 0.34673991, 0.48885077, 0.5348651 ,
       0.08436905, 0.02071519, 0.18776387, 0.52188862, 0.85832073,
       0.50843456, 0.60499752, 0.84728409, 0.93295583, 0.00486189,
       0.66121463, 0.3157777 , 0.6534353 , 0.12888288, 0.54033186,
       0.44024959, 0.37952039, 0.0833081 , 0.55721317, 0.36309742,
       0.2762448 , 0.46041519, 0.00914704, 0.35695435, 0.45490402])